In [4]:
import os
from openai import OpenAI
from pinecone import Pinecone

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index(os.getenv("PINECONE_INDEX_NAME"))

EMBED_MODEL = "text-embedding-3-large"   # 3072
CHAT_MODEL = "gpt-4.1-mini"

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
def embed_query(text: str):
    res = client.embeddings.create(
        model=EMBED_MODEL,
        input=text
    )
    return res.data[0].embedding


def search_docs(query: str, top_k=8, min_score=0.45):
    v = embed_query(query)
    res = index.query(
        vector=v,
        top_k=top_k,
        include_metadata=True
    )
    return [m for m in res["matches"] if m["score"] >= min_score]

In [ ]:
RAG_RULES = """
너는 오직 제공된 [근거]만으로 답한다.
- 근거에 없는 사실/법리/절차/판례/조문을 절대 추가하지 마라.
- 결론은 근거 문장을 최대한 그대로 재구성해라(새 표현 최소화).
- 반드시 근거 문장을 따옴표로 2~3개 인용해라.
- 불확실하면 '근거 부족'이라고 말해라.
"""

def answer_with_rag(query: str, return_matches=False):
    matches = search_docs(query)
    if not matches:
        return ("근거 부족: 관련 문서를 찾지 못했습니다.", matches) if return_matches else "근거 부족: 관련 문서를 찾지 못했습니다."

    context = "\n\n".join(
        f"[근거 {i+1} | score={m['score']:.3f}] {m['metadata'].get('text','')}"
        for i, m in enumerate(matches)
    )

    prompt = f"""{RAG_RULES}



[근거]
{context}

[질문]
{query}

[출력 형식]
1) 결론
2) 근거 인용(따옴표 2~3개)
3) 요약(3줄 이내)
4) 근거 부족한 부분(있으면)
"""

    res = client.chat.completions.create(
        model=CHAT_MODEL,
        temperature=0,
        messages=[{"role": "user", "content": prompt}]
    )
    return res.choices[0].message.content

In [ ]:
print(answer_with_rag(
    "임대인이 전세사기 재판 중이거나 주택에 압류·가압류가 있거나 경매 진행 중이면 계약 해지가 가능한가?"
))

In [7]:
# pip install -U python-dotenv openai pinecone langchain-text-splitters

In [9]:
import os
from dotenv import load_dotenv

from openai import OpenAI
from pinecone import Pinecone

from langchain_text_splitters import RecursiveCharacterTextSplitter


# ---------------------------
# 0) 환경 로드
# ---------------------------
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")

if not OPENAI_API_KEY or not PINECONE_API_KEY or not PINECONE_INDEX_NAME:
    raise ValueError("OPENAI_API_KEY / PINECONE_API_KEY / PINECONE_INDEX_NAME 를 .env에 설정해줘.")

client = OpenAI(api_key=OPENAI_API_KEY)

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)


# ---------------------------
# 1) 임베딩 유틸
# ---------------------------
def embed_text(text: str) -> list[float]:
    resp = client.embeddings.create(
        model="text-embedding-3-large",  # ✅ 3072 dim
        input=text
    )
    return resp.data[0].embedding


# ---------------------------
# 2) 문서 -> 청크 -> 업서트
# ---------------------------
def upsert_document(
    doc_id: str,
    text: str,
    namespace: str = "default",
    chunk_size: int = 800,
    chunk_overlap: int = 120,
):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""],
    )
    chunks = splitter.split_text(text)

    vectors = []
    for i, chunk in enumerate(chunks):
        vec = embed_text(chunk)
        vectors.append({
            "id": f"{doc_id}::chunk{i}",
            "values": vec,
            "metadata": {
                "doc_id": doc_id,
                "chunk_id": i,
                "text": chunk,
            }
        })

    # 배치 업서트
    index.upsert(vectors=vectors, namespace=namespace)
    return {"doc_id": doc_id, "chunks": len(chunks)}


# ---------------------------
# 3) 검색
# ---------------------------
def retrieve(
    query: str,
    namespace: str = "default",
    top_k: int = 5,
):
    qvec = embed_text(query)
    res = index.query(
        vector=qvec,
        top_k=top_k,
        include_metadata=True,
        namespace=namespace,
    )

    contexts = []
    for m in res.matches or []:
        md = m.metadata or {}
        contexts.append({
            "score": m.score,
            "doc_id": md.get("doc_id"),
            "chunk_id": md.get("chunk_id"),
            "text": md.get("text", ""),
        })
    return contexts


# ---------------------------
# 4) LLM 응답 (RAG)
# ---------------------------
SYSTEM = """너는 계약서/정책 문서를 검토하는 실무형 어시스턴트다.
주어진 근거(context) 안에서만 답하고, 근거가 부족하면 '근거 부족'이라고 말한다.
출력은 한국어로, 핵심 리스크/근거/권고조치 형태로 정리한다.
"""

def answer_with_rag(
    question: str,
    namespace: str = "default",
    top_k: int = 5,
    model: str = "gpt-4o-mini",
):
    ctxs = retrieve(question, namespace=namespace, top_k=top_k)

    context_block = "\n\n".join(
        [f"[doc={c['doc_id']} chunk={c['chunk_id']} score={c['score']:.3f}]\n{c['text']}"
         for c in ctxs]
    ) or "(검색 결과 없음)"

    prompt = f"""아래 context를 근거로 질문에 답해줘.

[context]
{context_block}

[question]
{question}

[output format]
- 리스크 요약(한 줄)
- 주요 리스크(불릿 3~7개)
  - 근거: (doc/chunk 인용)
  - 영향
  - 권고 조치
- 추가로 확인할 질문(있다면)
"""

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": prompt},
        ],
        temperature=0.2,
    )

    return {
        "contexts": ctxs,
        "answer": resp.choices[0].message.content
    }


# ---------------------------
# 5) 데모 실행
# ---------------------------
if __name__ == "__main__":
    sample_doc = """
    본 계약의 손해배상 한도는 연간 계약 금액의 100%로 한다.
    단, 고의 또는 중과실, 비밀유지 위반, 지식재산권 침해의 경우 손해배상 한도를 적용하지 않는다.
    계약 해지는 30일 전 서면 통지로 가능하다.
    """

    print("1) 업서트 중...")
    print(upsert_document(doc_id="contract_v1", text=sample_doc, namespace="contracts"))

    q = "손해배상 한도 조항에서 우리에게 불리한 리스크가 뭐야?"
    print("\n2) RAG 답변:\n")
    out = answer_with_rag(q, namespace="contracts", top_k=5)
    print(out["answer"])


1) 업서트 중...
{'doc_id': 'contract_v1', 'chunks': 1}

2) RAG 답변:

- 리스크 요약: 손해배상 한도가 연간 계약 금액의 100%로 제한되어 있어, 특정 상황에서 무한 책임이 발생할 수 있다.

- 주요 리스크
  - 근거: "본 계약의 손해배상 한도는 연간 계약 금액의 100%로 한다."
  - 영향: 계약 금액을 초과하는 손해가 발생할 경우, 추가적인 손해에 대한 보상이 이루어지지 않아 재정적 부담이 클 수 있다.
  - 권고 조치: 손해배상 한도를 조정하거나, 특정 상황에서의 무한 책임을 명확히 규정하는 조항을 추가하는 것을 검토해야 한다.

  - 근거: "단, 고의 또는 중과실, 비밀유지 위반, 지식재산권 침해의 경우 손해배상 한도를 적용하지 않는다."
  - 영향: 고의 또는 중과실로 인한 손해가 발생할 경우, 손해배상 한도가 적용되지 않아 무한 책임이 발생할 수 있다.
  - 권고 조치: 고의 또는 중과실의 정의를 명확히 하고, 이에 대한 책임 범위를 구체적으로 규정하는 것이 필요하다.

- 추가로 확인할 질문: 손해배상 한도를 조정할 수 있는 협상 가능성이 있는가?
